In [ ]:
from dataset import EcoPerceiverLoaderConfig, EcoPerceiverDataset
import torch
import numpy as np
import time
from tqdm import tqdm
import sqlite3
from torch.utils.data import DataLoader
from model import EcoSage
from components import EcoSageConfig
from torch.optim import adamw

In [ ]:
sites = ['US-Whs', 'CA-Gro', 'DE-Geb', 'CN-Din']
config = EcoPerceiverLoaderConfig(context_window_length=64, targets=['NEE'])
ds = EcoPerceiverDataset('/data/fluxes/carbonsense_v2', config)
dl = DataLoader(ds, batch_size=64, shuffle=True, collate_fn=ds.collate_fn, num_workers=8)

In [ ]:
config = EcoSageConfig()
model = EcoSage(config)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
optim = adamw.AdamW(params=model.parameters(), lr=3e-4)
print('Model created successfully')

In [ ]:
losses = []
for i, batch in enumerate(tqdm(dl)):
    optim.zero_grad()
    op, loss = model(batch)
    losses.append(loss.item())
    loss.backward()
    optim.step()
    if i == 100000:
        break

In [ ]:
model(batch, debug=True)

In [ ]:
# with sqlite3.connect('/data/fluxes/carbonsense_v2/carbonsense_v2.sql') as conn:
#     res =conn.execute('SELECT DISTINCT(igbp) FROM site_data;').fetchall()
# [r[0] for r in res]